In [ ]:
import angr
proj = angr.Project("/home/e1ec30/workspace/code/bayan/bayan/libtiff/my_build/bin/tiffcp", load_options={"auto_load_libs":False})
entry = proj.factory.entry_state(args=["tiffcp", "/tmp/poc", "/tmp/foo"])

In [ ]:
poc_raw = open("../poc", "rb").read()
poc = angr.SimFile("/tmp/poc", content=poc_raw, concrete=True, writable=False, has_end=True)
out = angr.SimFile("/tmp/foo")
entry.fs.insert("/tmp/poc", poc)
entry.fs.insert("/tmp/foo", out)

In [ ]:
strlen = angr.SIM_PROCEDURES["libc"]["strlen"]
exit = angr.procedures.libc.exit.exit

class quit_hook(exit):
    def run(self, *args):
        super().run(0)
        

class strlen_hook(strlen):
    def run(self, string):
        print(self.solver.symbolic(string))
        ret = super().run(string)
        return ret
    
class getopt_hook(angr.sim_procedure.SimProcedure):
    def run(self, *args):
        #we don't need getopt
        return -1


In [ ]:
proj.hook_symbol("getopt", getopt_hook())
proj.hook_symbol("strlen", strlen_hook())

In [ ]:
import logging
l = logging.getLogger('angr.engines.unicorn_engine').setLevel("DEBUG")
#l.setLevel("ERROR")

In [ ]:
sim = proj.factory.simgr(entry)

In [ ]:
class TimeoutError(Exception): # So we can limit execution time
    pass

def handle_timeout(signum, frame):
    raise TimeoutError("Timeout")

In [ ]:
import signal, time
def run_with_timeout(func, timeout, *args, **kwargs):
    signal.signal(signal.SIGALRM, handle_timeout)

    begin_time = time.time()

    signal.alarm(timeout)
    try:
        result = func(*args, **kwargs)

    except TimeoutError:
        #print(f"{func} timed out")
        result = None
        pass

    finally:
        end_time = time.time()
        total_time = int(end_time - begin_time)
        signal.alarm(0)

    return total_time, result

In [ ]:
run_with_timeout(sim.run, 1 * 60)

In [110]:
sim.one_active.solver.constraints

[<Bool unconstrained_ret_TIFFOpen_616_64{UNINITIALIZED} != 0x0>,
 <Bool unconstrained_ret_TIFFOpen_618_64{UNINITIALIZED} != 0x0>,
 <Bool unconstrained_ret_TIFFGetField_620_64{UNINITIALIZED}[31:0] == 0x0>,
 <Bool unconstrained_ret_TIFFGetField_622_64{UNINITIALIZED}[31:0] == 0x0>,
 <Bool unconstrained_ret_TIFFGetField_625_64{UNINITIALIZED}[31:0] == 0x0>,
 <Bool unconstrained_ret_TIFFGetField_629_64{UNINITIALIZED}[31:0] == 0x0>,
 <Bool unconstrained_ret_TIFFGetField_641_64{UNINITIALIZED}[31:0] == 0x0>,
 <Bool mem_7fffffffffefece_660_16{UNINITIALIZED} == 0x7>,
 <Bool unconstrained_ret_TIFFGetField_786_64{UNINITIALIZED}[31:0] == 0x0>,
 <Bool unconstrained_ret_TIFFGetField_920_64{UNINITIALIZED}[31:0] == 0x0>,
 <Bool unconstrained_ret_TIFFIsTiled_1611_64{UNINITIALIZED}[31:0] == 0x0>,
 <Bool unconstrained_ret_TIFFGetField_2023_64{UNINITIALIZED}[31:0] == 0x0>]

In [ ]:
from helpers import hash_state
st = set()

In [ ]:
for s in sim.deadended:
    if hash_state(s) not in st:
        try:
            print(hex(s.addr), ":", proj.loader.find_symbol(s.addr).name)
        except:
            pass
        for cs in s.callstack:
            try:
                print(f"{hex(cs.func_addr)} : {proj.loader.find_symbol(cs.func_addr).name}")
            except Exception as e:
                continue
        st.add(hash_state(s))
        print()
